In [1]:
#!git clone https://github.com/PAWSLabUniversityOfPittsburgh/Concept-Extraction.git

Cloning into 'Concept-Extraction'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 266 (delta 57), reused 259 (delta 52), pack-reused 0
Receiving objects: 100% (266/266), 13.98 MiB | 16.47 MiB/s, done.
Resolving deltas: 100% (57/57), done.


# Example for just one case

In [80]:
import os
import pandas as pd

path = "./Concept-Extraction/IIR-dataset/annotation"

In [81]:
csv_file_name = os.listdir(path)[0]
df = pd.read_csv(os.path.join(path, csv_file_name))
df.head()

,Concepts,Annotator 1,Annotator 2,Annotator 3
0,"['language models', 'language model']",1,1,1
1,['information retrieval'],1,1,1
2,"['queries', 'query']",1,1,1
3,['relevant document'],1,1,1
4,['language modeling approach'],1,1,1


In [82]:
df["sum"] = df.iloc[:,1:].sum(axis = 1)
df.head()

,Concepts,Annotator 1,Annotator 2,Annotator 3,sum
0,"['language models', 'language model']",1,1,1,3
1,['information retrieval'],1,1,1,3
2,"['queries', 'query']",1,1,1,3
3,['relevant document'],1,1,1,3
4,['language modeling approach'],1,1,1,3


In [83]:
# "['A','B','C','D']" -> ['A','B','C','D']
df["Concepts"] = df["Concepts"].apply(lambda x: ast.literal_eval(x))

In [88]:
csv_keywords_df = pd.DataFrame(columns = ["csv_file", "keywords"])

keywords = []
for row in df.itertuples():
    if row.sum >= 2:
        keywords += row.Concepts


csv_keywords_df.loc[len(csv_keywords_df)] = [csv_file_name, keywords]

In [101]:
csv_keywords_df.head()

,csv_file,keywords
0,iir-12.1.1.csv,"[language models, language model, information ..."


# Real Code

In [69]:
def correct_error(error_df):
    error_df_keywords = []
    for row in error_df.itertuples():
        if row.sum >= 2:
            try:
                error_df_keywords += ast.literal_eval(row.Concepts)
            except:
                print(f"{row.Concepts} has error, so it can be just ignored")
    
    return error_df_keywords

In [86]:
import os
import pandas as pd
import ast

path = "./Concept-Extraction/IIR-dataset/annotation"
csv_keywords_df = pd.DataFrame(columns = ["csv_file", "keywords"]) # final result
error_csv_files = []

for csv_file_name in os.listdir(path):
    
    # df
    df = pd.read_csv(os.path.join(path, csv_file_name))
    df["sum"] = df.iloc[:,1:].sum(axis = 1)
    keywords = []

    try:
        df["Concepts"] = df["Concepts"].apply(lambda x: ast.literal_eval(x))
        for row in df.itertuples():
            if row.sum >= 2:
                keywords += row.Concepts
    except:
        print(f"Error in {csv_file_name}")
        keywords = correct_error(df)
    
    keywords = list(map(lambda x: x.title(), keywords)) # 'language models' -> 'Language Models'
    
    csv_keywords_df.loc[len(csv_keywords_df)] = [csv_file_name, keywords]

Error in iir-9.1.csv
['nonrelevant document's vector'] has error, so it can be just ignored


In [89]:
csv_keywords_df.to_csv("csv_keywords_df.csv", index = False)

In [90]:
pd.read_csv("csv_keywords_df.csv")

,csv_file,keywords
0,iir-12.1.1.csv,"['Language Models', 'Language Model', 'Informa..."
1,iir-4.6.csv,"['Nonpositional Indexes', 'Nonpositional Index..."
2,iir-6.csv,"['Scoring', 'Term Weighting', 'Vector Space Mo..."
3,iir-14.2.csv,"['Rocchio Classification', 'Decision Boundarie..."
4,iir-8.1.csv,"['Document Collection', 'Information Needs', '..."
...,...,...
81,iir-13.1.csv,"['Text Classification Problem', 'Document Spac..."
82,iir-1.4.csv,"['Boolean Retrieval Model', 'Ranked Retrieval ..."
83,iir-11.2.csv,"['Probability Ranking Principle', 'Collection ..."
84,iir-4.1.csv,"['Index Construction', 'Inverted Index', 'Inde..."
